## <center> Applied Data Science Capstone</center>

this notebook will be mainly used for the Applied Data Science Capstone project.
Week 3: Segmenting and Clustering Neighborhoods in Toronto

## Table of Contents

<div class="alert alert-block alert-info" style="margin-top: 20px">

<font size = 3>

1. <a href="#item1">Part 1: Building the dataframe</a>

2. <a href="#item2">Part 2: utilize the Foursquare location data</a>

3. <a href="#item3">Part 3: Explore and cluster the neighborhoods in Toronto</a>

4. <a href="#item4">Part 4: Cluster Neighborhoods</a>

5. <a href="#item5">Part 5: Examine Clusters</a>    
</font>
</div>

<a id='item1'></a>

### Part 1: Building the dataframe

Before we get the data and start exploring it, let's download all the dependencies that we will need.

In [47]:
!conda install -c anaconda numpy
!conda install -c anaconda beautifulsoup4
!conda install -c anaconda lxml
!conda install -c anaconda requests
!conda install -c conda-forge geocoder
!conda install -c conda-forge folium=0.5.0 --yes
!conda install -c conda-forge geopy --yes

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - numpy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    openssl-1.1.1              |       h7b6447c_0         5.0 MB  anaconda
    certifi-2019.9.11          |           py36_0         154 KB  anaconda
    mkl-service-2.3.0          |   py36he904b0f_0         208 KB  anaconda
    numpy-base-1.16.5          |   py36hde5b4d6_0         4.4 MB  anaconda
    ca-certificates-2019.8.28  |                0         132 KB  anaconda
    numpy-1.16.5               |   py36h7e9f1db_0          49 KB  anaconda
    mkl-2019.5                 |              281       205.3 MB  anaconda
    ------------------------------------------------------------
                                           Total:       215.2 MB

The following packages will be UPDATED:

    certifi:         2

In [48]:
import pandas as pd # providing high-performance, easy-to-use data structures and data analysis tools
import numpy as np # library to handle data in a vectorized manner
import geocoder # import geocoder
import requests # for making HTTP requests in Python
import matplotlib.cm as cm # Builtin colormaps, colormap handling utilities
import matplotlib.colors as colors # A module for converting numbers or color arguments to RGB or RGBA
import folium # map rendering library

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
from sklearn.cluster import KMeans # The KMeans cluster algorithm
from bs4 import BeautifulSoup # for pulling data out of HTML and XML files

get wikipedia url, use BeautifulSoup for web scraping 

In [50]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
response = requests.get(url)

soup = BeautifulSoup(response.text, 'lxml')
table = soup.find('table', attrs={'class':'wikitable'})

table_rows = table.find_all('tr')

res = []
for tr in table_rows:
    td = tr.find_all('td')
    row = [tr.text.strip() for tr in td if tr.text.strip()]
    if row:
        res.append(row)


df_TorontoPostal = pd.DataFrame(res, columns=["Postcode", "Borough", "Neighborhood"])


In [51]:
# df_TorontoPostal.info()
df_TorontoPostal.head()

,Postcode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


do the needed clean up
- Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.
- Combined into 1 row where More than one neighborhood exist in one postal
- Borough for "Not assigned" neighborhood

In [53]:
df_TorontoPostal.drop(df_TorontoPostal[df_TorontoPostal['Borough'] == 'Not assigned'].index, inplace = True) 
df_TorontoPostal = df_TorontoPostal.groupby(['Postcode','Borough'])['Neighborhood'].apply(', '.join).reset_index()
df_TorontoPostal['Neighborhood'].loc[df['Neighborhood'] == 'Not assigned'] = df_TorontoPostal['Borough']

In [54]:
# display df_TorontoPostal 
df_TorontoPostal.head()

,Postcode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


Display final dataframe shape

In [55]:
df_TorontoPostal.shape

(103, 3)

<a id='item2'></a>

### Part 2: utilize the Foursquare location data

In [ ]:
# df_TorontoPostal["Latitude"] = np.nan
# df_TorontoPostal["Longitude"] = np.nan

In [ ]:
# # initialize your variable to None
# lat_lng_coords = None
# print(df.iterrows())
# for i,row in df.iterrows():
#     # loop until you get the coordinates
#     while(lat_lng_coords is None):
#         g = geocoder.google('{}, Toronto, Ontario'.format(df.at[i,'Postcode']))
#         lat_lng_coords = g.latlng
#         print(df.at[i,'Postcode'])
#     df.at[i,'Latitude'] = lat_lng_coords[0]
#     df.at[i,'Longitude'] = lat_lng_coords[1]

In [56]:
df_TorontoPostalCoords = pd.read_csv('https://cocl.us/Geospatial_data')

#rename "Postal Code" to "Postcode"
df_TorontoPostalCoords.rename(columns={'Postal Code':'Postcode'},inplace=True)

In [57]:
# display df_TorontoPostalCoords 
df_TorontoPostalCoords.head()

,Postcode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [58]:
# display df_TorontoPostal 
df_TorontoPostal.head()

,Postcode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [59]:
# Merge df_TorontoPostal and df_TorontoPostalCoords
df_Toronto = pd.merge(df_TorontoPostal, df_TorontoPostalCoords, on='Postcode')

In [60]:
# display df_Toronto
df_Toronto.head()

,Postcode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


<a id='item3'></a>

### Part 3: Explore and cluster the neighborhoods in Toronto

Use geopy library to get the latitude and longitude values of Totonto.

In [61]:
address = 'Toronto, ON, Canada'

geolocator = Nominatim(user_agent="to_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude

print(latitude, longitude)

43.653963 -79.387207


In [62]:
# create map of New York using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df_Toronto['Latitude'], df_Toronto['Longitude'], df_Toronto['Borough'], df_Toronto['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_newyork

In [63]:
# get Downtown Toronto
df_DowntownToronto = df_Toronto[df_Toronto['Borough'] == 'Downtown Toronto'].reset_index(drop=True)

In [64]:
# display df_DowntownToronto
df_DowntownToronto.head()

,Postcode,Borough,Neighborhood,Latitude,Longitude
0,M4W,Downtown Toronto,Rosedale,43.679563,-79.377529
1,M4X,Downtown Toronto,"Cabbagetown, St. James Town",43.667967,-79.367675
2,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
3,M5A,Downtown Toronto,"Harbourfront, Regent Park",43.654260,-79.360636
4,M5B,Downtown Toronto,"Ryerson, Garden District",43.657162,-79.378937


In [66]:
DowntownTorontAddress = 'Downtown Toront, ON, Canada'

geolocator = Nominatim(user_agent="to_explorer")
location = geolocator.geocode(address)
DowntownToront_lat = location.latitude
DowntownToront_long = location.longitude

print(DowntownToront_lat, DowntownToront_long)

43.653963 -79.387207


Use Foursquare library to get the latitude and longitude values of Totonto.

In [67]:
CLIENT_ID = '*******************' # your Foursquare ID
CLIENT_SECRET = '******************' # your Foursquare Secret
VERSION = '20180604' # Foursquare API version

# print('Your credentails:')
# print('CLIENT_ID: ' + CLIENT_ID)
# print('CLIENT_SECRET:' + CLIENT_SECRET)

In [69]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [70]:
def getNearbyVenues(names, latitudes, longitudes, radius=500,limit=100):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
#         print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            limit)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'NLatitude', 
                  'NLongitude', 
                  'Venue', 
                  'VLatitude', 
                  'VLongitude', 
                  'VCategory']
    
    return(nearby_venues)

In [71]:
df_DowntownToronto_venues = getNearbyVenues(names=df_DowntownToronto['Neighborhood'],
                                   latitudes=df_DowntownToronto['Latitude'],
                                   longitudes=df_DowntownToronto['Longitude']
                                  )

In [72]:
# Display df_DowntownToronto_venues
df_DowntownToronto_venues.head()

,Neighborhood,NLatitude,NLongitude,Venue,VLatitude,VLongitude,VCategory
0,Rosedale,43.679563,-79.377529,Mooredale House,43.678631,-79.380091,Building
1,Rosedale,43.679563,-79.377529,Rosedale Park,43.682328,-79.378934,Playground
2,Rosedale,43.679563,-79.377529,Whitney Park,43.682036,-79.373788,Park
3,Rosedale,43.679563,-79.377529,Alex Murray Parkette,43.678300,-79.382773,Park
4,Rosedale,43.679563,-79.377529,Milkman's Lane,43.676352,-79.373842,Trail


In [102]:
Map_DowntownToronto = folium.Map(location=[DowntownToront_lat, DowntownToront_long], zoom_start=14)

for vlat, vlng, venue, Category in zip(df_DowntownToronto_venues['VLatitude'], df_DowntownToronto_venues['VLongitude'], df_DowntownToronto_venues['Venue'],df_DowntownToronto_venues['VCategory']):
    label = '{}, {}'.format(venue,Category)
    label2 = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
    [vlat, vlng],
    radius=5,
    popup=label2,
    color='blue',
    fill=True,
    fill_color='#3186cc',
    fill_opacity=0.7,
    parse_html=False).add_to(Map_DowntownToronto)

Map_DowntownToronto

In [78]:
df_DowntownToronto_venues.groupby('Neighborhood').count()

,NLatitude,NLongitude,Venue,VLatitude,VLongitude,VCategory
Neighborhood,,,,,,
"Adelaide, King, Richmond",100,100,100,100,100,100
Berczy Park,57,57,57,57,57,57
"CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara",15,15,15,15,15,15
"Cabbagetown, St. James Town",47,47,47,47,47,47
Central Bay Street,85,85,85,85,85,85
"Chinatown, Grange Park, Kensington Market",100,100,100,100,100,100
Christie,15,15,15,15,15,15
Church and Wellesley,91,91,91,91,91,91
"Commerce Court, Victoria Hotel",100,100,100,100,100,100


In [79]:
print('There are {} uniques categories.'.format(len(df_DowntownToronto_venues['VCategory'].unique())))

There are 207 uniques categories.


In [82]:
# one hot encoding
DowntownToronto_onehot = pd.get_dummies(df_DowntownToronto_venues[['VCategory']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
DowntownToronto_onehot['Neighborhood'] = df_DowntownToronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [DowntownToronto_onehot.columns[-1]] + list(DowntownToronto_onehot.columns[:-1])
DowntownToronto_onehot = DowntownToronto_onehot[fixed_columns]

DowntownToronto_onehot.head()

,Yoga Studio,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Theater,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0


In [84]:
DowntownToronto_onehot.shape

(1296, 207)

In [85]:
DowntownToronto_grouped = DowntownToronto_onehot.groupby('Neighborhood').mean().reset_index()
DowntownToronto_grouped

,Neighborhood,Yoga Studio,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Theater,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint
0,"Adelaide, King, Richmond",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.030000,...,0.010000,0.000000,0.00,0.0,0.00,0.020000,0.000000,0.000000,0.010000,0.000000
1,Berczy Park,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.00,0.0,0.00,0.017544,0.000000,0.000000,0.000000,0.000000
2,"CN Tower, Bathurst Quay, Island airport, Harbo...",0.000000,0.000000,0.066667,0.066667,0.066667,0.133333,0.133333,0.066667,0.000000,...,0.000000,0.000000,0.00,0.0,0.00,0.000000,0.000000,0.000000,0.000000,0.000000
3,"Cabbagetown, St. James Town",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.00,0.0,0.00,0.000000,0.000000,0.000000,0.000000,0.000000
4,Central Bay Street,0.011765,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.011765,...,0.000000,0.000000,0.00,0.0,0.00,0.011765,0.000000,0.000000,0.011765,0.000000
5,"Chinatown, Grange Park, Kensington Market",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.01,0.0,0.00,0.060000,0.000000,0.040000,0.010000,0.000000
6,Christie,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.00,0.0,0.00,0.000000,0.000000,0.000000,0.000000,0.000000
7,Church and Wellesley,0.010989,0.010989,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.010989,...,0.010989,0.010989,0.00,0.0,0.00,0.000000,0.010989,0.010989,0.000000,0.010989
8,"Commerce Court, Victoria Hotel",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.040000,...,0.000000,0.000000,0.00,0.0,0.00,0.020000,0.000000,0.000000,0.010000,0.000000
9,"Design Exchange, Toronto Dominion Centre",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.030000,...,0.010000,0.000000,0.00,0.0,0.01,0.010000,0.000000,0.000000,0.010000,0.000000


In [86]:
DowntownToronto_grouped.shape

(18, 207)

In [87]:
num_top_venues = 5

for hood in DowntownToronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = DowntownToronto_grouped[DowntownToronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide, King, Richmond----
            venue  freq
0     Coffee Shop  0.08
1            Café  0.05
2      Steakhouse  0.04
3             Bar  0.04
4  Cosmetics Shop  0.03


----Berczy Park----
                venue  freq
0         Coffee Shop  0.07
1        Cocktail Bar  0.05
2  Italian Restaurant  0.04
3      Farmers Market  0.04
4            Beer Bar  0.04


----CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara----
             venue  freq
0   Airport Lounge  0.13
1  Airport Service  0.13
2    Boat or Ferry  0.07
3  Harbor / Marina  0.07
4         Boutique  0.07


----Cabbagetown, St. James Town----
         venue  freq
0  Coffee Shop  0.09
1   Restaurant  0.06
2          Pub  0.04
3       Market  0.04
4       Bakery  0.04


----Central Bay Street----
                venue  freq
0         Coffee Shop  0.14
1  Italian Restaurant  0.05
2                Café  0.05
3      Ice Cream Shop  0.04
4        Burger Joint  0.04


---

In [88]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [126]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = DowntownToronto_grouped['Neighborhood']

for ind in np.arange(DowntownToronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(DowntownToronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide, King, Richmond",Coffee Shop,Café,Steakhouse,Bar,Hotel,Thai Restaurant,Burger Joint,American Restaurant,Cosmetics Shop,Restaurant
1,Berczy Park,Coffee Shop,Cocktail Bar,Café,Cheese Shop,Farmers Market,Beer Bar,Bakery,Steakhouse,Italian Restaurant,Seafood Restaurant
2,"CN Tower, Bathurst Quay, Island airport, Harbo...",Airport Lounge,Airport Service,Harbor / Marina,Bar,Plane,Boutique,Sculpture Garden,Boat or Ferry,Coffee Shop,Airport
3,"Cabbagetown, St. James Town",Coffee Shop,Restaurant,Market,Italian Restaurant,Bakery,Pizza Place,Café,Pub,Chinese Restaurant,Butcher
4,Central Bay Street,Coffee Shop,Italian Restaurant,Café,Middle Eastern Restaurant,Chinese Restaurant,Sandwich Place,Burger Joint,Ice Cream Shop,Bar,Sushi Restaurant


<a id='item4'></a>

### Part 4: Cluster Neighborhoods

In [127]:
# set number of clusters
kclusters = 5

DowntownToronto_grouped_clustering = DowntownToronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(DowntownToronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 


array([0, 0, 3, 0, 0, 2, 4, 0, 0, 0], dtype=int32)

In [128]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'ClusterLabels', kmeans.labels_)

DowntownToronto_merged = df_DowntownToronto

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
DowntownToronto_merged = DowntownToronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

# toronto_merged['ClusterLabels'] = toronto_merged['ClusterLabels'].astype(int)

DowntownToronto_merged.head() # check the last columns!

,Postcode,Borough,Neighborhood,Latitude,Longitude,ClusterLabels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4W,Downtown Toronto,Rosedale,43.679563,-79.377529,1,Park,Playground,Trail,Building,Department Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant,Dog Run
1,M4X,Downtown Toronto,"Cabbagetown, St. James Town",43.667967,-79.367675,0,Coffee Shop,Restaurant,Market,Italian Restaurant,Bakery,Pizza Place,Café,Pub,Chinese Restaurant,Butcher
2,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160,0,Coffee Shop,Japanese Restaurant,Gay Bar,Sushi Restaurant,Restaurant,Gym,Café,Fast Food Restaurant,Mediterranean Restaurant,Men's Store
3,M5A,Downtown Toronto,"Harbourfront, Regent Park",43.654260,-79.360636,0,Coffee Shop,Bakery,Pub,Café,Park,Gym / Fitness Center,Breakfast Spot,Restaurant,Theater,Mexican Restaurant
4,M5B,Downtown Toronto,"Ryerson, Garden District",43.657162,-79.378937,0,Coffee Shop,Clothing Store,Middle Eastern Restaurant,Cosmetics Shop,Café,Japanese Restaurant,Ramen Restaurant,Restaurant,Plaza,Bakery


In [129]:
# create map
map_clusters = folium.Map(location=[DowntownToront_lat, DowntownToront_long], zoom_start=13)


# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(DowntownToronto_merged['Latitude'], DowntownToronto_merged['Longitude'], DowntownToronto_merged['Neighborhood'], DowntownToronto_merged['ClusterLabels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

<a id='item5'></a>

## 5. Examine Clusters

#### Cluster 1

In [144]:
DowntownToronto_merged.loc[DowntownToronto_merged['ClusterLabels'] == 0, DowntownToronto_merged.columns[[2] + list(range(5, DowntownToronto_merged.shape[1]))]]

,Neighborhood,ClusterLabels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,"Cabbagetown, St. James Town",0,Coffee Shop,Restaurant,Market,Italian Restaurant,Bakery,Pizza Place,Café,Pub,Chinese Restaurant,Butcher
2,Church and Wellesley,0,Coffee Shop,Japanese Restaurant,Gay Bar,Sushi Restaurant,Restaurant,Gym,Café,Fast Food Restaurant,Mediterranean Restaurant,Men's Store
3,"Harbourfront, Regent Park",0,Coffee Shop,Bakery,Pub,Café,Park,Gym / Fitness Center,Breakfast Spot,Restaurant,Theater,Mexican Restaurant
4,"Ryerson, Garden District",0,Coffee Shop,Clothing Store,Middle Eastern Restaurant,Cosmetics Shop,Café,Japanese Restaurant,Ramen Restaurant,Restaurant,Plaza,Bakery
5,St. James Town,0,Coffee Shop,Café,Hotel,Restaurant,Italian Restaurant,Clothing Store,Bakery,Beer Bar,Cocktail Bar,Cosmetics Shop
6,Berczy Park,0,Coffee Shop,Cocktail Bar,Café,Cheese Shop,Farmers Market,Beer Bar,Bakery,Steakhouse,Italian Restaurant,Seafood Restaurant
7,Central Bay Street,0,Coffee Shop,Italian Restaurant,Café,Middle Eastern Restaurant,Chinese Restaurant,Sandwich Place,Burger Joint,Ice Cream Shop,Bar,Sushi Restaurant
8,"Adelaide, King, Richmond",0,Coffee Shop,Café,Steakhouse,Bar,Hotel,Thai Restaurant,Burger Joint,American Restaurant,Cosmetics Shop,Restaurant
9,"Harbourfront East, Toronto Islands, Union Station",0,Coffee Shop,Hotel,Aquarium,Café,Scenic Lookout,Fried Chicken Joint,Brewery,Sports Bar,Baseball Stadium,Bar
10,"Design Exchange, Toronto Dominion Centre",0,Coffee Shop,Café,Hotel,Restaurant,Deli / Bodega,Bakery,Italian Restaurant,Gastropub,Bar,American Restaurant


#### Cluster 2

In [145]:
DowntownToronto_merged.loc[DowntownToronto_merged['ClusterLabels'] == 1, DowntownToronto_merged.columns[[2] + list(range(5, DowntownToronto_merged.shape[1]))]]

,Neighborhood,ClusterLabels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Rosedale,1,Park,Playground,Trail,Building,Department Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant,Dog Run


#### Cluster 3

In [146]:
DowntownToronto_merged.loc[DowntownToronto_merged['ClusterLabels'] == 2, DowntownToronto_merged.columns[[2] + list(range(5, DowntownToronto_merged.shape[1]))]]

,Neighborhood,ClusterLabels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
12,"Harbord, University of Toronto",2,Café,Restaurant,Bookstore,Sandwich Place,Japanese Restaurant,Bar,Bakery,Chinese Restaurant,Pub,Nightclub
13,"Chinatown, Grange Park, Kensington Market",2,Café,Vegetarian / Vegan Restaurant,Bar,Dumpling Restaurant,Chinese Restaurant,Bakery,Mexican Restaurant,Vietnamese Restaurant,Coffee Shop,Comfort Food Restaurant


#### Cluster 4

In [147]:
DowntownToronto_merged.loc[DowntownToronto_merged['ClusterLabels'] == 3, DowntownToronto_merged.columns[[2] + list(range(5, DowntownToronto_merged.shape[1]))]]

,Neighborhood,ClusterLabels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
14,"CN Tower, Bathurst Quay, Island airport, Harbo...",3,Airport Lounge,Airport Service,Harbor / Marina,Bar,Plane,Boutique,Sculpture Garden,Boat or Ferry,Coffee Shop,Airport


#### Cluster 5

In [149]:
DowntownToronto_merged.loc[DowntownToronto_merged['ClusterLabels'] == 4, DowntownToronto_merged.columns[[2] + list(range(5, DowntownToronto_merged.shape[1]))]]

,Neighborhood,ClusterLabels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
17,Christie,4,Grocery Store,Café,Park,Coffee Shop,Nightclub,Diner,Italian Restaurant,Baby Store,Restaurant,Convenience Store
